In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from astartes import train_val_test_split

# Read in the data
- This csv file was directly taken from [Zenodo](https://zenodo.org/record/6618262#.Y-ZRzMHMLUI) which stores data from the following publication: Kevin A. Spiekermann, Lagnajit Pattanaik, and William H. Green. "High Accuracy Barrier Heights, Enthalpies, and Rate Coefficients for Chemical Reactions". In: Sci. Data 9.1 (2022), pp. 1–12. [link](https://www.nature.com/articles/s41597-022-01529-6)




In [3]:
CSV_PATH = 'ccsdtf12_dz.csv'
df = pd.read_csv(CSV_PATH)
df

,idx,rsmi,psmi,dE0,dHrxn298,rmg_family
0,0,[C:1]([c:2]1[n:3][o:4][n:5][n:6]1)([H:7])([H:8...,[C:1]([C:2]([N:3]=[O:4])=[N+:6]=[N-:5])([H:7])...,48.61085,26.77621,NaN
1,1,[C:1]([c:2]1[n:3][o:4][n:5][n:6]1)([H:7])([H:8...,[C:1]([N:3]=[C:2]=[N:6][N:5]=[O:4])([H:7])([H:...,74.02980,28.79099,NaN
2,2,[C:1]([O:2][C:3]([C:4]([O:5][H:13])([H:11])[H:...,[C:1]1([H:6])([H:7])[O:2][C:3]([H:9])([H:10])[...,97.42200,12.60220,NaN
3,3,[C:1]([O:2][C:3]([C:4]([O:5][H:13])([H:11])[H:...,[C:1]([O:2][H:13])([H:6])([H:7])[H:8].[C:3]1([...,75.25375,28.98589,NaN
4,4,[C:1]([O:2][C:3]([C:4]([O:5][H:13])([H:11])[H:...,[C:1]([O:2][H:13])([H:6])([H:7])[H:8].[C:3]([C...,72.16356,1.41779,NaN
...,...,...,...,...,...,...
11921,11956,[C:1]([C@@:2]([O:3][H:12])([C:4]([O:5][C:6](=[...,[C:1]([C:2][C:4]([O:5][C:6](=[O:7])[H:15])([H:...,75.56813,79.63518,NaN
11922,11957,[C:1]([C@@:2]([O:3][H:12])([C:4]([O:5][C:6](=[...,[C:1]([C@@:2]1([H:11])[O:3][C@:6]([O:7][H:12])...,42.41621,5.79695,NaN
11923,11958,[C:1]([C@@:2]([O:3][H:12])([C:4]([O:5][C:6](=[...,[C:1]([C@@:2]([O:3][H:12])([C:4](=[O:5])[H:14]...,72.75039,30.54744,NaN
11924,11959,[C:1]([C@@:2]([O:3][H:12])([C:4]([O:5][C:6](=[...,[C:1](=[C:2]([C:4]([O:5][C:6](=[O:7])[H:15])([...,65.83112,14.48350,"1,3_Insertion_ROR"


In [4]:
df.describe()

,idx,dE0,dHrxn298
count,11926.000000,11926.000000,11926.000000
mean,5974.624266,80.060894,35.416565
std,3449.834665,21.849569,30.213470
min,0.000000,9.418760,-113.992160
25%,2989.250000,65.051600,14.471478
50%,5972.500000,78.670340,33.097140
75%,8955.750000,93.110327,58.510087
max,11960.000000,195.598150,172.165720


# Featurize the data using morgan fingerprint (ECFP4)
- Use default settings of 2048 hash length with radius 2
- To represent the reaction, we will concatenate the reactant fingerprint with the difference of the product and reactant fingerprint, which is reminiscient of the established condensed graph of reaction representation

Define helper function taken from Chemprop

https://github.com/chemprop/chemprop/blob/master/chemprop/features/features_generators.py

In [5]:
MORGAN_RADIUS = 2
MORGAN_NUM_BITS = 2048
def morgan_counts_features_generator(mol,
                                     radius= MORGAN_RADIUS,
                                     num_bits= MORGAN_NUM_BITS):
    """
    Generates a counts-based Morgan fingerprint for a molecule.
    :param mol: A molecule (i.e., either a SMILES or an RDKit molecule).
    :param radius: Morgan fingerprint radius.
    :param num_bits: Number of bits in Morgan fingerprint.
    :return: A 1D numpy array containing the counts-based Morgan fingerprint.
    """
    mol = Chem.MolFromSmiles(mol) if type(mol) == str else mol
    features_vec = AllChem.GetHashedMorganFingerprint(mol, radius, nBits=num_bits)
    features = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(features_vec, features)

    return features

In [6]:
params = Chem.SmilesParserParams()
params.removeHs = False

In [7]:
r_fp = []
p_fp = []
for i, row in df.iterrows():
    rsmi, psmi = row.rsmi, row.psmi
    
    rmol = Chem.MolFromSmiles(rsmi, params)
    morgan = morgan_counts_features_generator(rmol)
    r_fp.append(morgan)
    
    pmol = Chem.MolFromSmiles(psmi, params)
    morgan = morgan_counts_features_generator(rmol)
    p_fp.append(morgan)

In [8]:
df_r_fp = pd.DataFrame(r_fp)
df_p_fp = pd.DataFrame(p_fp)

In [9]:
# check the dimensions
df_r_fp.shape

(11926, 2048)

# Train a LinearSVR model using random splits

To keep this example concise and focus on demonstrating the syntax for using the astartes package, we knowingly omit many standard steps from an ML workflow such as
- Using the validation set to identify good hyperparameters
- Z-scoring the regression targets before training

Note that LinearSVR was chosen for its speed, not its accuracy for this task. The input representation is chosen to be a concatenation of the morgan fingerprint of the reactant with the difference in morgan fingerprint between the product complex and the reactant. This is reminciscient of the established Condensded Graph of Reaction (CGR) representation that has been described and used in the following sample publications:
- Alexandre Varnek et al. “Substructural Fragments: An Universal Language to Encode Reactions, Molecular and Supramolecular Structures”. In: J. Comput.-Aided Mol. Des. 19.9 (2005), pp. 693–703. [link](https://link.springer.com/article/10.1007/s10822-005-9008-0)
- Frank Hoonakker et al. “Condensed Graph of Reaction: Considering a Chemical Reaction as One Single Pseudo Molecule”. In: Int. J. Artif. Intell. Tools 20.2 (2011), pp. 253–270. [link](https://dtai.cs.kuleuven.be/events/ilp-mlg-srl/USBStick/papers/ILP09-5.pdf)
- Esther Heid and William H. Green. “Machine Learning of Reaction Properties via Learned Representations of the Condensed Graph of Reaction”. In: J. Chem. Inf. Model. 62.9 (2021), pp. 2101–2110. [link](https://pubs.acs.org/doi/full/10.1021/acs.jcim.1c00975)
- Kevin A. Spiekermann, Lagnajit Pattanaik, and William H. Green. “Fast Predictions of Reaction Barrier Heights: Toward Coupled-Cluster Accuracy”. In: J. Phys. Chem. A 126.25 (2022), pp. 3976–3986. [link](https://pubs.acs.org/doi/full/10.1021/acs.jpca.2c02614)


In [10]:
def train_model(seed, sampler, hopts={}, df_r_fp=df_r_fp):
    # use random splits to create 85:5:10 data split
    train_indices, val_indices, test_indices = train_val_test_split(df_r_fp.values,
                                                                    train_size=0.85,
                                                                    val_size=0.05,
                                                                    test_size=0.1,
                                                                    sampler=sampler,
                                                                    random_state=seed,
                                                                    hopts=hopts,
                                                                    return_indices=True,
                                                                   )    
    y_train = df.dE0.values[train_indices]
    y_val   = df.dE0.values[val_indices]
    y_test  = df.dE0.values[test_indices]
    
    X_train = np.concatenate((df_r_fp.values[train_indices],
                              df_p_fp.values[train_indices] - df_r_fp.values[train_indices]),
                             axis=1)
        
    X_val   = np.concatenate((df_r_fp.values[val_indices],
                              df_p_fp.values[val_indices] - df_r_fp.values[val_indices]),
                              axis=1)
    
    X_test  = np.concatenate((df_r_fp.values[test_indices],
                              df_p_fp.values[test_indices] - df_r_fp.values[test_indices]),
                              axis=1)
    
    # use default hyperparameters
    regressor_obj = LinearSVR()
    regressor_obj.fit(X_train, y_train)
    y_pred = regressor_obj.predict(X_train)
    
    # training error
    train_mae = mean_absolute_error(y_train, y_pred)
    print(f'Train MAE: {train_mae :.2f} kcal/mol')
    
    train_rmse = mean_squared_error(y_train, y_pred, squared=False)
    print(f'Train RMSE: {train_rmse:.2f} kcal/mol\n')

    # validation error
    y_pred = regressor_obj.predict(X_val)
    val_mae = mean_absolute_error(y_val, y_pred)
    print(f'Val MAE: {val_mae :.2f} kcal/mol')
    
    val_rmse = mean_squared_error(y_val, y_pred, squared=False)
    print(f'Val RMSE: {val_rmse :.2f} kcal/mol\n')
    
    # testing error
    y_pred = regressor_obj.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: {test_mae :.2f} kcal/mol')
    
    test_rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(f'Test RMSE: {test_rmse :.2f} kcal/mol\n')
    
    
    # include a baseline model that simply predicts the mean value from the training set
    y_baseline = [y_train.mean()] * len(y_test)
    baseline_test_mae = mean_absolute_error(y_test, y_baseline)
    print(f'Baseline model test MAE: {baseline_test_mae :.2f} kcal/mol')
    
    baseline_test_rmse = mean_squared_error(y_test, y_baseline, squared=False)
    print(f'Baseline model test RMSE: {baseline_test_rmse :.2f} kcal/mol\n')
    
    return test_mae, test_rmse, baseline_test_mae, baseline_test_rmse

In [11]:
test_maes, test_rmses = [], []
baseline_test_maes, baseline_test_rmses = [], []

for seed in range(5):
    print('*'*44)
    print(f'seed {seed}')
    test_mae, test_rmse, baseline_test_mae, baseline_test_rmse = train_model(seed,
                                                                             sampler='random',
                                                                            )
    test_maes.append(test_mae)
    test_rmses.append(test_rmse)
    
    baseline_test_maes.append(baseline_test_mae)
    baseline_test_rmses.append(baseline_test_rmse)

********************************************
seed 0
Train MAE: 14.88 kcal/mol
Train RMSE: 19.90 kcal/mol

Val MAE: 17.05 kcal/mol
Val RMSE: 21.95 kcal/mol

Test MAE: 16.39 kcal/mol
Test RMSE: 21.49 kcal/mol

Baseline model test MAE: 17.33 kcal/mol
Baseline model test RMSE: 22.58 kcal/mol

********************************************
seed 1
Train MAE: 14.87 kcal/mol
Train RMSE: 19.90 kcal/mol

Val MAE: 16.54 kcal/mol
Val RMSE: 21.41 kcal/mol

Test MAE: 16.88 kcal/mol
Test RMSE: 21.70 kcal/mol

Baseline model test MAE: 17.63 kcal/mol
Baseline model test RMSE: 22.20 kcal/mol

********************************************
seed 2
Train MAE: 14.92 kcal/mol
Train RMSE: 19.99 kcal/mol

Val MAE: 16.72 kcal/mol
Val RMSE: 21.23 kcal/mol

Test MAE: 16.34 kcal/mol
Test RMSE: 21.07 kcal/mol

Baseline model test MAE: 17.24 kcal/mol
Baseline model test RMSE: 21.65 kcal/mol

********************************************
seed 3
Train MAE: 14.94 kcal/mol
Train RMSE: 19.93 kcal/mol

Val MAE: 16.70 kcal/mol


In [12]:
print(f'Test MAE (mean +- 1 std): {np.mean(test_maes):.3f} +- {np.std(test_maes):.3f} kcal/mol')
print(f'Test RMSE (mean +- 1 std): {np.mean(test_rmses):.3f} +- {np.std(test_rmses):.3f} kcal/mol')

Test MAE (mean +- 1 std): 16.438 +- 0.225 kcal/mol
Test RMSE (mean +- 1 std): 21.230 +- 0.313 kcal/mol


As a baseline, use a trivial model that only predicts the mean value of the training set. The LinearSVR model shown above barely outperforms this trivial baseline.

In [13]:
print(f'Baseline model test MAE (mean +- 1 std): {np.mean(baseline_test_maes):.3f} +- {np.std(baseline_test_maes):.3f} kcal/mol')
print(f'Baseline model test RMSE (mean +- 1 std): {np.mean(baseline_test_rmses):.3f} +- {np.std(baseline_test_rmses):.3f} kcal/mol')

Baseline model test MAE (mean +- 1 std): 17.221 +- 0.259 kcal/mol
Baseline model test RMSE (mean +- 1 std): 21.883 +- 0.457 kcal/mol
